In [ ]:
%%capture
%pip install yfinance
%pip install pysentiment2
%pip install transformers
%pip install py-readability-metrics
%python -m nltk.downloader punkt

In [1]:
import yfinance as yf
import pysentiment2 as ps
from bs4 import BeautifulSoup
import requests
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from datetime import datetime as dt
import spacy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import MinMaxScaler
import torch
from readability import Readability
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [3]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
from numpy import loadtxt
file = open('batch2.csv', 'rb')
data = np.loadtxt(file, delimiter=",",dtype='str')

df = pd.DataFrame(data)

mask = (df == 'nan').any(1)
df = df[~mask] #gets rid of all rows with string 'nan'
data = df.to_numpy()
print(data)

[['MMM' '2021' '4' ... '11.140535015026959' '34.04855319621302'
  '70.70256731749184']
 ['MMM' '2021' '3' ... '10.793553405168563' '38.32631380425198'
  '69.42418092957224']
 ['MMM' '2021' '1' ... '12.68835289967788' '44.57263896515957'
  '66.4627367720271']
 ...
 ['ZTS' '2015' '2' ... '15.112257680678326' '43.25211681196191'
  '45.4412662531803']
 ['ZTS' '2015' '1' ... '13.454273218885966' '40.4055555429274'
  '55.35886017280515']
 ['ZTS' '2014' '4' ... '12.68835289967788' '46.02614538683838'
  '58.878374642147904']]


In [5]:
data[0,:]

array(['MMM', '2021', '4', '-0.04189472305504671', '-0.14615007454974582',
       '-0.15708868256537967', '-0.13270961824249328',
       '-0.12258565616124262', '-0.046017860200147975',
       '-0.14982455628852887', '-0.16071609079135551',
       '-0.1364419400182546', '-0.12636146743188026',
       '0.8189584853752043', '95.0', '69.0', '0.15853658439916718',
       '0.08291203231399796', '101.0', '66.0', '0.2095808370683782',
       '0.0709732256391954', '14.317249209551537', '7.378095800032188',
       '18.112190935396296', '11.051034253014397', '16.404322709996244',
       '11.140535015026959', '34.04855319621302', '70.70256731749184'],
      dtype=object)

Dataset Split

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X_data = data[:,13:]
Y_data = data[:,5] 
#Y_data = data[:,5] looks at difference between Day 0 and Day 50
X_data = X_data.astype(np.float64)
Y_data = Y_data.astype(np.float64)

#labels data
# classify >5% as 1
# classify 5 to -5% as 0 
# classify <5% as -1
df = pd.DataFrame(Y_data)

df.loc[(df[0] >= -0.10)] = 0
df.loc[(df[0] < -0.10)] = 1

# df.loc[((df[0] < 0.4) & (df[0] >= 0.03))] = 1
# df.loc[((df[0] < -0.03) & (df[0] >= -0.4))] = -1
# df.loc[((df[0] < 0.05) & (df[0] >= -0.05))] = 0


Y_data = df.to_numpy()


#scales the X datasets within a range of 0 to 1
scaler = MinMaxScaler()
X_data = scaler.fit_transform(X_data)


#splits the dataset into 67% training and 33% test
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.33) 

REGRESSION

In [283]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

ml = LinearRegression()
ml.fit(X_train, Y_train)

Y_pred = ml.predict(X_test)
ml1_score = ml.score(X_test, Y_test)
ml1_r2_score = r2_score(Y_test, Y_pred)

print(ml1_score)
print(ml1_r2_score) 

df = pd.DataFrame({'Actual':Y_test, 'Predicted': Y_pred.flatten()})
df.head(50)

TypeError: 'numpy.float64' object is not callable

In [39]:
ml2 = RandomForestRegressor(max_depth=2, random_state=0)
ml2.fit(X_train, Y_train)

Y_pred = ml2.predict(X_test)
score = ml2.score(X_test, Y_test)


print(score)


df = pd.DataFrame({'Actual':Y_test, 'Predicted': Y_pred.flatten()})
df.head(50)

NameError: name 'RandomForestRegressor' is not defined

In [ ]:
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr = MLPRegressor(random_state=1, max_iter=1000).fit(X_train, Y_train)


CLASSIFICATION

In [57]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier
from sklearn import metrics

rfc = RandomForestClassifier(max_depth=1000, n_estimators=1000)

rfc.fit(X_train, Y_train)
score = rfc.score(X_test, Y_test)
Y_pred = rfc.predict(X_test)
 
print(metrics.classification_report(Y_test, Y_pred))  

/var/folders/dr/g848f5b55mn9xp6p982hbg280000gn/T/ipykernel_35202/2355902801.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train, Y_train)


              precision    recall  f1-score   support

         0.0       0.91      1.00      0.95      5455
         1.0       0.82      0.02      0.03       578

    accuracy                           0.91      6033
   macro avg       0.86      0.51      0.49      6033
weighted avg       0.90      0.91      0.86      6033



In [17]:
df = pd.DataFrame({'Actual':Y_test.flatten(), 'Predicted': Y_pred.flatten()})
df.head(30)

,Actual,Predicted
0,1.0,1.0
1,1.0,1.0
2,1.0,1.0
3,0.0,1.0
4,0.0,0.0
5,-1.0,1.0
6,0.0,0.0
7,1.0,1.0
8,-1.0,1.0
9,-1.0,1.0


In [121]:
Y_pred = rfc.predict_proba(X_test)
df = pd.DataFrame({'probabilityA':Y_pred[:,0], 'probabilityB':Y_pred[:,1], 'probabilityC':Y_pred[:,2], 'probabilityD':Y_pred[:,3],  'probabilityE':Y_pred[:,4]})
df.head(30)

,probabilityA,probabilityB,probabilityC,probabilityD,probabilityE
0,0.011,0.181,0.443,0.350,0.015
1,0.008,0.247,0.400,0.313,0.032
2,0.079,0.220,0.342,0.286,0.073
3,0.013,0.187,0.496,0.293,0.011
4,0.143,0.200,0.236,0.307,0.114
5,0.010,0.158,0.528,0.293,0.011
6,0.017,0.144,0.481,0.319,0.039
7,0.041,0.146,0.418,0.377,0.018
8,0.045,0.174,0.472,0.282,0.027
9,0.005,0.154,0.528,0.296,0.017


In [155]:
# SVC(kernel="linear", C=0.025),
# SVC(kernel="poly", degree=3, C=0.025),
# SVC(kernel="rbf", C=1, gamma=2),
# GaussianProcessClassifier(1.0 * RBF(1.0)),
# GradientBoostingClassifier(n_estimators=100, learning_rate=1.0),
# DecisionTreeClassifier(max_depth=5),
# ExtraTreesClassifier(n_estimators=10, min_samples_split=2),
# RandomForestClassifier(max_depth=5, n_estimators=100),
# MLPClassifier(alpha=1, max_iter=1000),
# AdaBoostClassifier(n_estimators=100),
# GaussianNB(),
# QuadraticDiscriminantAnalysis(),
# SGDClassifier(loss="hinge", penalty="l2")]
clf = (
GradientBoostingClassifier(n_estimators=100, learning_rate=1.0)
)
clf.fit(X_train, Y_train)
score = clf.score(X_test, Y_test)
Y_pred = clf.predict(X_test)
 
print(metrics.classification_report(Y_test, Y_pred))  

/Users/victor/miniforge3/envs/NLP_env/lib/python3.8/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
df = pd.DataFrame({'Actual':Y_test.flatten(), 'Predicted': Y_pred.flatten()})
df.head(60)

In [3]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier


names = ["Nearest_Neighbors", "Linear_SVM", "Polynomial_SVM", "RBF_SVM", "Gaussian_Process",
         "Gradient_Boosting", "Decision_Tree", "Extra_Trees", "Random_Forest", "Neural_Net", "AdaBoost",
         "Naive_Bayes", "QDA", "SGD"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(kernel="poly", degree=3, C=0.025),
    SVC(kernel="rbf", C=1, gamma=2),
    GradientBoostingClassifier(n_estimators=100, learning_rate=1.0),
    DecisionTreeClassifier(max_depth=5),
    ExtraTreesClassifier(n_estimators=10, min_samples_split=2),
    RandomForestClassifier(max_depth=5, n_estimators=100),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(n_estimators=100),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
    SGDClassifier(loss="hinge", penalty="l2")]


scores = []
for name, clf in zip(names, classifiers):
    clf.fit(X_train, Y_train)
    score = clf.score(X_test, Y_test)
    scores.append(score)




NameError: name 'X_train' is not defined

In [ ]:
scores

In [102]:
Y_pred = clf.predict(X_test)
Y_pred

array([1., 1., 1., ..., 1., 1., 1.])

In [103]:
r2_score(Y_test, Y_pred)

-0.7898685409196073

In [104]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn import metrics
from sklearn.metrics import eval

print("MAE:",mean_absolute_error(Y_test, Y_pred))
print("MSE:",mean_squared_error(Y_test, Y_pred))
print("RMSE:",mean_squared_error(Y_test, Y_pred,squared=False))
print("R2:",r2_score(Y_test, Y_pred))
#print("RMSLE:",mean_squared_log_error( y_test, y_pred,squared=False))
print("MAPE:",mean_absolute_percentage_error(Y_test, Y_pred))

print(metrics.classification_report(Y_test, Y_pred))

MAE: 0.7514636032254501
MSE: 1.2876394565337457
RMSE: 1.1347420220181086
R2: -0.7898685409196073
MAPE: 935244371971339.5
              precision    recall  f1-score   support

        -1.0       0.30      0.06      0.10      2371
         0.0       0.19      0.01      0.02      1896
         1.0       0.53      0.95      0.68      4786

    accuracy                           0.52      9053
   macro avg       0.34      0.34      0.26      9053
weighted avg       0.40      0.52      0.39      9053



In [107]:
loss, acc = clf.evaluate(Y_test, Y_pred)


AttributeError: 'RandomForestClassifier' object has no attribute 'evaluate'